# clean.py  
**Authors:** Arjun Sai Krishnan, Carina Lewandowski  
**Description:** series of functions for preprocessing hCoV sequences.

In [ ]:
# import modules from biopython
from Bio import SeqIO, Seq

## Remove Extraneous Sequences
This chunk removes sequences that contain non-real bases or do not meet the length threshold.

In [3]:
# ---------------------------------------------------------------------------
# REMOVE EXTRANEOUS SEQUENCES 
# ---------------------------------------------------------------------------
def remove_extr_seqs(records): 
    sequences = []
    is_bad = False
    count = 0
    threshold = 29000 # length threshold

    # iterate over each record & parse its sequence as a string
    for record in records:
        curr_seq = str(record.seq)
        # trim white space for the purpose of inspecting base characters
        curr_seq.replace(' ', '')
        curr_seq.replace('\n', '')
        # check that seq length meets threshold
        if len(curr_seq) < threshold:
            is_bad = True
        # visit each position to check for extraneous base; break and reomve
        # sequence once one is found.
        for char in curr_seq:
            if char != 'A' and char != 'C' and char != 'T' and char !='G':
                print(char)
                count = count + 1
                print(count)
                is_bad = True
                break
        # append sequence to list if it is not extraneous
        if not is_bad: 
            sequences.append(record)
        # reset boolean check
        is_bad = False
        
    # write non-extraneous sequences to a new text file in fasta format
    SeqIO.write(sequences, "cleaned.txt", "fasta")
    SeqIO.write(records[0], "test.txt", "fasta")

## Remove Gaps
After alignment with MAFFT, gaps typically appear in the largest amounts at the beginning and ends of sequences. This chunk finds the right number of bases to trim from the beginning and ends of sequences.

In [ ]:
# ---------------------------------------------------------------------------
# REMOVE GAPS
# ---------------------------------------------------------------------------
def remove_gaps(records):
    # iterate over each record & parse its sequence as a string
    for record in records:
        curr_seq = str(record.seq)
        # find length of consecutive gaps at the beginning of the seq
        i = 0
        max_gaps_beg = 0
        while(curr_seq[i] == '-'):
            i += 1
        # find length of consecutive gaps at the end of the seq
        j = -1
        max_gaps_end = 0
        while(curr_seq[j] == '-'):
            j -= 1
        # updated max lengths
        if (i > max_gaps_beg): max_gaps_beg = i
        if (-1*j > max_gaps_end): max_gaps_end = j

    updated_records = []
    # iterate over each record and trim front and back of sequences
    for record in records:
        new_record = record
        new_seq = str(record.seq)[(max_gaps_beg - 1):(max_gaps_end + 1)]
        new_seq = Seq.Seq(new_seq)
        new_record.seq = new_seq
        updated_records.append(new_record)

    # write trimmed records to a new text file in fasta format
    SeqIO.write(updated_records, "aligned_cleaned.txt", "fasta")
    SeqIO.write(records[0], "test_aligned.txt", "fasta")
    print("Trimmed from front: ", i)
    print("Trimmed from back: ", j)

## Find Segregating Sites

In [4]:
# ---------------------------------------------------------------------------
# FIND SEGREGATING SITES
# ---------------------------------------------------------------------------
def find_seg_sites(records):
    site_dict = {}
    # iterate over each record & parse its sequence as a string
    for record in records:
        curr_seq = str(record.seq)
        # iterate over each position in the current seq
        for i in range(len(curr_seq)):
            base = curr_seq[i]
            # create a dict for each seq position
            # (key: position, value: dict of bases)
            if (i not in site_dict):
                site_dict[i] = {}
            # create a dict for each base found at each position
            # (key: base, value: frequency)
            if (base not in site_dict[i]):
                site_dict[i][base] = 0    
            site_dict[i][base] += 1
    
    # if more than one base dict exists at a position, classify as a seg site
    seg_sites = []
    for site in site_dict:
        if (len(site_dict[site]) > 1):
            seg_sites.append((site, site_dict[site]))

    return seg_sites

## Testing

In [5]:
# ---------------------------------------------------------------------------
# TESTING
# ---------------------------------------------------------------------------

''' TESTING remove_extr_seqs '''
# records = list(SeqIO.parse("gisaid_hcov-19_India_2020_11_11_15.txt", "fasta"))
# records = list(SeqIO.parse("test.txt", "fasta"))

''' TESTING remove_gaps '''
# records2 = list(SeqIO.parse("aligned.txt", "fasta"))
# remove_gaps(records2)

''' TESTING find_seg_sites '''
# records3 = list(SeqIO.parse("aligned_cleaned.txt", "fasta"))
# seg_sites = find_seg_sites(records3)
# print("Number of segregating sites:", len(seg_sites))

' TESTING find_seg_sites '